# 前提条件

## ホストマシンに物理NICが3つあって、1つは上流に接続、2つはブリッジに繋がった状態であること

/etc/network/interfacesの例
```
auto lo
iface lo inet loopback

# 1つは上流に接続
auto eth0
iface eth0 inet dhcp

# 2つはブリッジに繋がった状態
auto enx14
iface enx14 inet manual

auto enx19
iface enx19 inet manual

auto br14
iface br14 inet static
address 192.168.14.1
netmask 255.255.255.0
bridge_ports enx14
bridge_maxwait 0
bridge_fd 0
bridge_stp off

auto br19
iface br19 inet static
address 192.168.19.1
netmask 255.255.255.0
bridge_ports enx19
bridge_maxwait 0
bridge_fd 0
bridge_stp off

```

jupiterは3つのネットワークを全て繋ぐ設定で起動しているので、192.168.14.0/24と192.168.19.0/24のネットワークに繋がったインタフェースが見えていることと、上流への疎通があってインターネットに出れることを確認しておく。

In [ ]:
!ip a
!test $(ip a | grep 192.168.14 | wc -l) -eq 1
!test $(ip a | grep 192.168.19 | wc -l) -eq 1

!ping -c 4 yahoo.jp
!test $(ping -c 4 yahoo.jp | grep "4 packets received" | wc -l) -eq 1

## 同梱のdocker-compose.ymlでコンテナ一式が起動した状態であること

[同梱のdocker-compose.yml](/edit/wifi/docker-compose.yml)

docker-composeで作成された各種リソースには、起動した場所（docker-compose.ymlの場所）を示す文字列が接頭語として追加される。このnotebookでは/rootで起動して、接頭語がrootになっている前提とする。

コンテナ起動状態を確認するため、dockerコマンドをインストール（jupyterには/var/run/docker.sockをマウントして、dockerコマンドを使えるようにしてある）し、juyter、vyos、muninのコンテナが動いていることと、dockerの内部ネットワーク、root_default、root_wifi、root_wireが存在することを確認しておく。

In [ ]:
!curl -fsSL https://download.docker.com/linux/static/stable/x86_64/docker-17.09.0-ce.tgz \
    | sudo tar -xzC /usr/local/bin --strip=1 docker/docker

!sudo docker ps
!test $(sudo docker ps | grep jupyter | wc -l) -eq 1
!test $(sudo docker ps | grep vyos | wc -l) -eq 1
!test $(sudo docker ps | grep munin | wc -l) -eq 1

!sudo docker network ls
!test $(sudo docker network ls | grep root_default | wc -l) -eq 1
!test $(sudo docker network ls | grep root_wifi | wc -l) -eq 1
!test $(sudo docker network ls | grep root_wire | wc -l) -eq 1

# Juputerの設定

タイムゾーンをJSTにする。dockerコマンドをインストールする。

この[niicloudoperation/notebook](https://hub.docker.com/r/niicloudoperation/notebook/)は、国立情報学研究所クラウド運用チームの研究実践「Literate Computing for Reproducible Infrastructure」で実際に使われているもので、SDNにOpen Flowを使ったベアメタルのプライベートクラウドの構築・運用に使われている。

In [ ]:
%%bash
sudo -i
echo Asia/Tokyo > /etc/timezone
dpkg-reconfigure --frontend noninteractive tzdata
date

# VyOSの設定

[vyos](http://wiki.vyos-users.jp/)には3つのネットワークを繋ぐが、docker-composeの挙動により、default以外のネットワークがコンテナにアタッチされる順番が保証されず、ネットワークとinterfaceの対応関係が固定されない（順序を固定する方法が見つからなかった）。そこで、wireのアタッチをコンテナ起動後に追加設定として行う。

因みに、docker createして、ネットワークをアタッチして、startさせる場合は、bridgeネットワークの名前順で認識される。dokcerの使われ方の主流では、デバイスとか気にしない感じで問題ないので、この辺の作りがイマイチなままなんだろうな。

vyosのinterfaceを確認する。起動時にアタッチされた2つ（eth0、eth1）が見えている。

In [ ]:
!sudo docker exec vyos ifconfig
!test $(sudo docker exec vyos ifconfig | grep eth0 | wc -l) -eq 1
!test $(sudo docker exec vyos ifconfig | grep eth1 | wc -l) -eq 1
!test $(sudo docker exec vyos ifconfig | grep eth2 | wc -l) -eq 0

dockerがコンテナ用に持っているネットワークを確認する。docker-composeで作ったネットワークには起動場所の接頭語が付くので注意。

In [ ]:
!sudo docker network ls

vyosにroot_wireをアタッチ。

In [ ]:
!sudo docker network connect root_wire vyos

3つ目のネットワークがeth2として増えているのを確認する。

In [ ]:
!sudo docker exec vyos ifconfig
!test $(sudo docker exec vyos ifconfig | grep eth0 | wc -l) -eq 1
!test $(sudo docker exec vyos ifconfig | grep eth1 | wc -l) -eq 1
!test $(sudo docker exec vyos ifconfig | grep eth2 | wc -l) -eq 1

vyosのルータ設定はansible経由で行う。connection:dockerで接続するため、vyos_configが使えないので、スクリプトファイルをコピーしてシェル実行する。併せて、docker execからvyosのコマンドを叩くためのスクリプトファイルもコピーしておく。

[vyosの設定](/edit/wifi/vyos-setup.vsh)

[コマンドのラッパ](/edit/wifi/run-command.sh)

In [ ]:
!sudo ansible-playbook -i vyos, vyos-setup.yml

こんな感じでvyosのコマンドが叩ける。

In [ ]:
!sudo docker exec --user vyos vyos /home/vyos/run-command.sh show interfaces

因みに、VyOSは実質Debianなので、interfaceが絡んだ機能（例えば、FireWallやQoS）は、OSのネットワーク設定で実現されている。このため、本事例のようにコンテナで利用すると、ネットワークデバイスが完全仮想化されていないことから、ネットワーク関連の機能のいくつかが動作しない。vyosにdocker execしてドヤ顔するのが不要で、そこそこのスペックのマシンが用意できるなら、普通にxenとか使った完全仮想化の方が、トラブルも少ないだろう。

# Aironetの設定

工場出荷時設定の[Aironet1140](https://www.cisco.com/c/ja_jp/td/docs/nma/works/works4windows/gs/001/ap1140aut-getstart.html)を2台、wifiのネットワークに繋ぐ。vyosの設定に、macアドレスに対して固定のIPアドレスを払い出すように書いてあるので、Aironetがそのアドレスを掴めたか確認する。起動にそこそこ時間がかかるので、時間を空けて何度か試すこと。

因みにAironet1140を工場出荷時設定に戻すには、「MODE」スイッチを押したまま電源を疎通させランプが赤点灯になるまで押し続けることで行える。

In [ ]:
!ping -c 4 ap64
!test $(ping -c 4 ap64 | grep "4 packets received" | wc -l) -eq 1
!ping -c 4 ap83
!test $(ping -c 4 ap83 | grep "4 packets received" | wc -l) -eq 1

工場出荷時設定のAironetは、telnetでデフォルトアカウントで入れるようになっているので、新規のアカウントで、sshでしか入れないように設定する。

[playbook](/edit/wifi/aironet-setup.yml)

In [ ]:
!ansible-playbook -i ap64,ap83, aironet-setup.yml -t ssh-setup

ansibleがAironetにsshするためにknown_hostsにfingerprintを覚えさせる必要がある。ansibleの設定で自動登録できるらしいが、jupyter内に入れてあるansibleのバージョンの問題か、動作しなかったので、ssh-keyscanで対応する。Aironetへのssh接続でfingerprintの確認が出ない状態にできたら、アクセスポイントの設定を行える。

In [ ]:
!mkdir -p ~/.ssh
!ssh-keyscan -H ap64 > ~/.ssh/known_hosts
!ssh-keyscan -H ap83 >> ~/.ssh/known_hosts

In [ ]:
!ansible-playbook -i ap64,ap83, aironet-setup.yml -t setup

# 接続の確認

wifiで繋いだり、有線で繋いだりして、インターネットへの接続が行えることを確認する。

# muninの設定

ネットワークの稼働をモニタするのに[munin](http://munin-monitoring.org/)を使う。

vyos、ap64、ap83をsnmpで監視する設定を行い、muninコンテナを再起動する。

In [ ]:
!sudo docker exec munin bash -c "munin-node-configure --snmp vyos --snmpversion 2c --snmpcommunity muninconnect --shell | sh"

In [ ]:
!sudo docker exec munin bash -c "munin-node-configure --snmp ap64 --snmpversion 2c --snmpcommunity muninconnect --shell | sh"

In [ ]:
!sudo docker exec munin bash -c "munin-node-configure --snmp ap83 --snmpversion 2c --snmpcommunity muninconnect --shell | sh"

In [ ]:
!sudo docker exec munin bash -c 'echo -e "\n[snmp_*]\nenv.community muninconnect\nenv.version 2\n" >> /etc/munin/plugin-conf.d/munin-node'

In [ ]:
!sudo docker exec munin bash -c 'echo -e "\n[vyos]\n    address 127.0.0.1\n    use_node_name no\n[ap64]\n    address 127.0.0.1\n    use_node_name no\n[ap83]\n    address 127.0.0.1\n    use_node_name no\n" >> /etc/munin/munin.conf'

In [ ]:
!sudo docker restart munin

直ぐにグラフを出せるように、cron1回分を先に実行しておく。

In [ ]:
!sleep 30s
!sudo docker exec --user munin munin /usr/bin/munin-cron

muninコンテナ80番ポートにブラウザでアクセスするとグラフが見れる。